In [80]:
import torch
from torch import nn
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import string 
import nltk
import seaborn as sns 
nltk.download('punkt')
from nltk.tokenize import word_tokenize
data = pd.read_excel("Data.xlsx")
def Tokenize(text):
    return (word_tokenize(text.lower()))
data["Tokenizing"]= data["tweet"].apply(lambda x : Tokenize(x))
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
def no_stop(text):
    text = [c for c in text if c not in stop_words]
    return text
data["not_stop"]= data["Tokenizing"].apply(lambda x : no_stop(x))

from nltk.stem import wordnet
from nltk.stem import WordNetLemmatizer
word_lem = WordNetLemmatizer()
def lemmatize(text):
    text = [word_lem.lemmatize(c) for c in text ]
    return text
data["lemmatize"]=data["not_stop"].apply(lambda x : lemmatize(x))
data.drop(["Tokenizing","not_stop"] ,axis='columns', inplace=True)
def clean(text):
    l =[]
    for c in text :
        c = re.sub('\[.*?\]','', c)
        c = re.sub("\\W","",c) 
        c = re.sub(r'https.+','', c, flags=re.MULTILINE)
        c = re.sub('<.*?>+','', c)
        c = re.sub('[%s]' % re.escape(string.punctuation),'', c)
        c = re.sub('\n','', c)
        c = re.sub('\w*\d\w*','', c)
        c = re.sub('[^a-zA-Z]','',c)
        l.append(c)
    for i in l:
        if i == '':
            l.remove(i)
        if i == ' ':
            l.remove(i)
        if i == '  ':
            l.remove(i) 
        if i == '   ':
            l.remove(i)  
    l = " ".join([c for c in l])

       
    return l    
data["clean"]=data["lemmatize"].apply(lambda x : clean(x))





from sklearn.feature_extraction.text import TfidfVectorizer
corpus=[]
for i in range (0, len(data)):
  corpus.append(data['clean'][i])
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=2,stop_words='english')
X = vectorizer.fit_transform(corpus)
a = vectorizer.get_feature_names()
def all_(text):
    text = Tokenize(text)
    text = no_stop(text)
    text = lemmatize(text)
    text = clean(text)
    return text
df =pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names())
df


[nltk_data] Downloading package punkt to /Users/macbook/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,aaiofficial,aaj,aajtak,aamaadmiparty,aamir,ababa,abandoned,abbott,abia,ability,...,zee,zero,zika,zinc,zithromax,zombie,zone,zoo,zookeepers,zoom
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6415,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6416,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6417,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6418,0.0,0.0,0.0,0.0,0.416388,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [81]:
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import tensorflow as tf
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
enc = preprocessing.OrdinalEncoder(categories='auto')
from sklearn.model_selection import StratifiedShuffleSplit

data["label"]
c = data["label"].to_numpy()
c = c.reshape(-1,1)
enc.fit(c)
b =enc.transform(c)
lab = pd.DataFrame(b , columns=["label"])
lab


,label
0,1.0
1,1.0
2,0.0
3,1.0
4,1.0
...,...
6415,0.0
6416,0.0
6417,0.0
6418,0.0


In [82]:
df_tot = pd.concat([df,lab],axis=1)

In [83]:
import torchvision
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class MyData(Dataset):
  def __init__(self,data,targets):
    super(MyData,self)
    self.data= data
    self.targets = targets
  def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]
  def __len__(self):
        return len(self.targets)
X=df_tot.iloc[:,:-1].values
Y=df_tot.iloc[:,-1].values

ss=StratifiedShuffleSplit(n_splits=1,test_size=0.4,random_state=31)
for train_index , group_index in ss.split(X,Y):
  Xtrain , Xgroup = X[train_index] , X[group_index]
  Ytrain , Ygroup = Y[train_index] , Y[group_index]
for test_index , val_index in ss.split(Xgroup,Ygroup):
  Xtest , Xval = Xgroup[test_index] , Xgroup[val_index]
  Ytest , Yval = Ygroup[test_index] , Ygroup[val_index]
sc = StandardScaler()
Xtrain = sc.fit_transform(Xtrain)
Xtest = sc.fit_transform(Xtest)
Xval =  sc.fit_transform(Xval)  


Xtrain,Xtest,Xval = map(torch.tensor,(Xtrain,Xtest,Xval))
Ytrain , Ytest,Yval = map(torch.tensor,(Ytrain,Ytest,Yval))

Xtrain=Xtrain.float()
Xtest = Xtest.float()
Xval=Xval.float()
Ytrain  =Ytrain.float()#Ytrain.type(torch.LongTensor)#Ytrain.float()
mydd = MyData(Xtrain,Ytrain)
train_loader = DataLoader(dataset = mydd, batch_size=1000)    

In [84]:
def compare(output , label):
    L=[]
    for i in output:
        if i >= 0.5:
            L.append(1)
        else :
            L.append(0)
    T = label.numpy() 
    t=0
    for i in range(len(T)):
        if T[i]==L[i]:
            t+=1
    return t/len(T)

     

In [85]:
import torch 
class NNNNN(nn.Module):
  def __init__ (self):
    super(NNNNN, self).__init__()
    self.l1=nn.Linear(5469 ,100)
    self.l2=nn.Linear(100,100)
    self.l3=nn.Linear(100,1)
    self.relu =nn.ReLU()
    self.sig=nn.Sigmoid()
    self.init_weights()
  def init_weights(self):
    torch.nn.init.kaiming_normal_(self.l1.weight)  
    torch.nn.init.kaiming_normal_(self.l2.weight)  
    torch.nn.init.kaiming_normal_(self.l3.weight)  

  def forward(self,x):
    out_int = self.l1(x)
    out_func = self.relu(out_int)
    out_func = self.l2(out_func)
    out_func = self.relu(out_func)
    out=self.l3(out_func)
    return out

In [86]:
nttt=NNNNN()



In [87]:

train_loader_ = DataLoader(dataset = mydd, batch_size=1000)
loss_ = torch.nn.BCELoss(reduction="mean")
loss_function = nn.CrossEntropyLoss()


optimizer = torch.optim.Adam(nttt.parameters(), lr=0.0003, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
epoch = 10
nttt.train()
T =[]
for ep in range(epoch):
    
    
    for feature , label in train_loader_:
        optimizer.zero_grad()
        
        output = nttt.forward(feature)
        
        loss = torch.nn.functional.binary_cross_entropy_with_logits(output,label.unsqueeze(1))

        loss.backward()
        
        optimizer.step()
        
    nttt.eval()
    predect = nttt.forward(Xval)
    t = compare(output,label)
    v=compare(predect,Yval)
    print("epoch {}, train_accuracy: {:.4f}, valid_accuracy: {:.4f}".format(ep,t,v ))



epoch 0, train_accuracy: 0.5423, valid_accuracy: 0.5778
epoch 1, train_accuracy: 0.7254, valid_accuracy: 0.6673
epoch 2, train_accuracy: 0.8462, valid_accuracy: 0.7335
epoch 3, train_accuracy: 0.9096, valid_accuracy: 0.7714
epoch 4, train_accuracy: 0.9413, valid_accuracy: 0.8035
epoch 5, train_accuracy: 0.9648, valid_accuracy: 0.8249
epoch 6, train_accuracy: 0.9765, valid_accuracy: 0.8356
epoch 7, train_accuracy: 0.9930, valid_accuracy: 0.8434
epoch 8, train_accuracy: 0.9953, valid_accuracy: 0.8473
epoch 9, train_accuracy: 0.9965, valid_accuracy: 0.8482


In [88]:
predectt = nttt(Xtest)
print(compare(predectt,Ytest))

0.85


In [89]:
def cleaner(tweet) : 
        text = all_(tweet)
        tfidf_encoding =vectorizer.transform([text])
        tfidf_encodings_array = tfidf_encoding.toarray()
        return tfidf_encodings_array
def predictt(text):
    with torch.no_grad():
       
       tes=cleaner(text)
       #tes=tes.todense()
       test= torch.tensor(tes,dtype=torch.float)
       output = nttt.forward(test)
       if output >= 0.5:
            c = 1
       else :
            c=0;
            
       return c 

In [ ]:
from tkinter import *

root = Tk()
root.geometry("600x600")
root.title(" DOcFN ")

def Take_input():
    INPUT = inputtxt.get("1.0", "end-1c")
    test = all_(INPUT)
    out = predictt(test)
    #print(out)
    if(out == 1):
        
        Output.insert(END, 'Real')
    else:
        
        Output.insert(END, 'Fake')

l = Label(text = "Predict your news 😍")
inputtxt = Text(root, height = 25,width = 50,bg = "light yellow",borderwidth=1,relief="solid")

Output = Text(root, height = 5,width = 25,bg = "light yellow",borderwidth=1,relief="solid")

Display = Button(root, height = 2,width = 20,text ="Show",command = lambda:Take_input())


l.pack()
inputtxt.pack()
Display.pack()
Output.pack()


mainloop()